Hyperparameter Tuning

In [3]:
import os
from datetime import datetime

import optuna
import torch
from lightning.pytorch import Trainer, seed_everything
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
from lightning.pytorch.loggers import TensorBoardLogger

from common.file_paths import BASE_DIR
from modules import (GenerateSchematicCallback,
                     LightningTransformerMinecraftStructureGenerator,
                     MinecraftDataModule)

torch.set_float32_matmul_precision('medium')


def objective(trial: optuna.Trial):
    seed_everything(0, workers=True)

    lightning_model = LightningTransformerMinecraftStructureGenerator(
        num_classes=250,
        max_sequence_length=512,
        embedding_dim=trial.suggest_categorical("embedding_dim", [16, 32, 64]),
        embedding_dropout=trial.suggest_float(
            "embedding_dropout", 0.1, 0.4, step=0.1),
        num_heads=trial.suggest_categorical("num_heads", [8, 16, 32]),
        num_layers=trial.suggest_int("num_layers", 2, 4, 6),
        decoder_dropout=trial.suggest_float(
            "decoder_dropout", 0.1, 0.4, step=0.1),
        freeze_encoder=True,
        learning_rate=trial.suggest_float(
            "learning_rate", 1e-5, 1e-3, log=True)
    )

    hdf5_file = os.path.join(BASE_DIR, 'data.h5')
    data_module = MinecraftDataModule(
        file_path=hdf5_file,
        batch_size=trial.suggest_categorical("batch_size", [32, 64, 128]),
    )

    logger = TensorBoardLogger(
        'lightning_logs', name='minecraft_structure_generator', log_graph=False)
    checkpoint_callback = ModelCheckpoint(
        monitor='val_loss',
        mode='min',
        save_last=True,
        save_weights_only=True
    )
    early_stop_callback = EarlyStopping(
        monitor='val_loss',
        patience=20,
        verbose=True,
        mode='min'
    )
    generate_schematic_callback = GenerateSchematicCallback(
        save_path='schematic_viewer/public/schematics/',
        data_module=data_module,
        generate_train=False,
        generate_val=True,
        generate_all_datasets=False,
        generate_every_n_epochs=10
    )

    trainer = Trainer(
        max_epochs=5000,
        logger=logger,
        gradient_clip_val=1.0,
        log_every_n_steps=5,
        callbacks=[
            checkpoint_callback,
            early_stop_callback,
            generate_schematic_callback
        ]
    )

    trainer.fit(lightning_model, datamodule=data_module)

    best_score = checkpoint_callback.best_model_score
    if best_score is not None:
        return best_score.item()
    elif "val_loss" in trainer.callback_metrics:
        return trainer.callback_metrics["val_loss"].item()
    else:
        return 0.25


study_name = "limited_blocks_spheres_increased_samples"
# study_name = f"study_{datetime.now().strftime('%Y%m%d%H%M%S')}"
study = optuna.create_study(direction='minimize', study_name=study_name,
                            storage='sqlite:///studies.db', load_if_exists=True)
study.optimize(objective, n_trials=100)

print("Best hyperparameters:", study.best_trial.params)

[I 2024-01-13 19:22:04,278] Using an existing study with name 'limited_blocks_spheres_increased_samples' instead of creating a new one.
Seed set to 0
c:\Users\mmmfr\Documents\Repositories\minecraft-schematic-generator\.venv\Lib\site-packages\optuna\distributions.py:700: UserWarning: The distribution is specified by [2, 4] and step=6, but the range is not divisible by `step`. It will be replaced by [2, 2].
  warnings.warn(
[W 2024-01-13 19:22:04,385] Trial 45 failed with parameters: {'embedding_dim': 64, 'embedding_dropout': 0.4, 'num_heads': 32, 'num_layers': 2, 'decoder_dropout': 0.1, 'learning_rate': 0.00033275115474582705} because of the following error: TypeError("LightningTransformerMinecraftStructureGenerator.__init__() got an unexpected keyword argument 'embedding_dim'").
Traceback (most recent call last):
  File "c:\Users\mmmfr\Documents\Repositories\minecraft-schematic-generator\.venv\Lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = f

In [1]:
import os

import torch
from lightning.pytorch import Trainer, seed_everything
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
from lightning.pytorch.loggers import TensorBoardLogger

from common.file_paths import BASE_DIR
from modules import (GenerateSchematicCallback,
                     LightningTransformerMinecraftStructureGenerator,
                     MinecraftDataModule)

torch.set_float32_matmul_precision('medium')

seed_everything(0, workers=True)

lightning_model = LightningTransformerMinecraftStructureGenerator(
    num_classes=10000,
    max_sequence_length=1331,
    embedding_dropout=0.2,
    model_dim=768,
    num_heads=12,
    num_layers=12,
    decoder_dropout=0.2,
    learning_rate=1e-4
)
# lightning_model = LightningTransformerMinecraftStructureGenerator.load_from_checkpoint(
#     'lightning_logs/version_55/checkpoints/last.ckpt')

hdf5_file = os.path.join(BASE_DIR, 'data.h5')
data_module = MinecraftDataModule(
    file_path=hdf5_file,
    batch_size=6,
    # num_workers=4
)

logger = TensorBoardLogger(
    'lightning_logs', name='minecraft_structure_generator', log_graph=False)
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    mode='min',
    save_last=True,
    save_weights_only=False,
    save_top_k=2
)
early_stop_callback = EarlyStopping(
    monitor='val_loss',
    patience=50,
    verbose=True,
    mode='min'
)
generate_schematic_callback = GenerateSchematicCallback(
    masked_path='schematic_viewer/public/schematics/masked/',
    filled_path='schematic_viewer/public/schematics/filled/',
    data_module=data_module,
    generate_train=False,
    generate_val=True,
    generate_all_datasets=False,
    generate_every_n_epochs=1,
    temperature=0.7
)

trainer = Trainer(
    max_epochs=5000,
    # profiler='simple',
    gradient_clip_val=1.0,
    log_every_n_steps=5,
    callbacks=[
        checkpoint_callback,
        early_stop_callback,
        generate_schematic_callback
    ]
)

# lightning_model.trainer = trainer

# # Set the new learning rate
# lightning_model.learning_rate = 1e-3  # Set the new learning rate

# # Recreate the optimizers and schedulers
# optimizers_and_schedulers = lightning_model.configure_optimizers()

# # Assign the recreated optimizers and schedulers to the trainer
# lightning_model.trainer.optimizers = [optimizers_and_schedulers['optimizer']]
# lightning_model.trainer.lr_schedulers = [optimizers_and_schedulers['lr_scheduler']]

trainer.fit(lightning_model, datamodule=data_module)

c:\Users\mmmfr\Documents\Repositories\minecraft-schematic-generator\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Seed set to 0
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                                   | Params
-----------------------------------------------------------------
0 | model | TransformerMinecraftStructureGenerator | 110 M 
-----------------------------------------------------------------
110 M     Trainable params
0         Non-trainable params
110 M     Total params
443.707   Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

c:\Users\mmmfr\Documents\Repositories\minecraft-schematic-generator\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.


c:\Users\mmmfr\Documents\Repositories\minecraft-schematic-generator\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.


...

Epoch 110: 100%|██████████| 621/621 [07:45<00:00,  1.33it/s, v_num=67]

Metric val_loss improved by 0.031 >= min_delta = 0.0. New best score: 0.644


Epoch 130: 100%|██████████| 621/621 [09:10<00:00,  1.13it/s, v_num=67]

Metric val_loss improved by 0.015 >= min_delta = 0.0. New best score: 0.629


Epoch 140:  34%|███▎      | 209/621 [02:56<05:47,  1.19it/s, v_num=67]

In [ ]:
import os

import torch
from lightning.pytorch import Trainer, seed_everything
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
from lightning.pytorch.loggers import TensorBoardLogger

from common.file_paths import BASE_DIR
from model import VAE
from modules import (GenerateSchematicCallback, MinecraftDataModule,
                     VAELightningModule)

torch.set_float32_matmul_precision('medium')

seed_everything(0, workers=True)

vae = VAE(num_classes=256)
lightning_model = VAELightningModule(vae, learning_rate=1e-3)

hdf5_file = os.path.join(BASE_DIR, 'data.h5')
data_module = MinecraftDataModule(
    file_path=hdf5_file,
    batch_size=32,
    # num_workers=4
)

logger = TensorBoardLogger(
    'lightning_logs', name='minecraft_structure_generator', log_graph=False)
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    mode='min',
    save_last=True,
    save_weights_only=False
)
early_stop_callback = EarlyStopping(
    monitor='val_loss',
    patience=20,
    verbose=True,
    mode='min'
)
generate_schematic_callback = GenerateSchematicCallback(
    save_path='schematic_viewer/public/schematics/',
    data_module=data_module,
    generate_train=False,
    generate_val=True,
    generate_all_datasets=False,
    generate_every_n_epochs=1
)

trainer = Trainer(
    max_epochs=5000,
    logger=logger,
    gradient_clip_val=1.0,
    log_every_n_steps=5,
    callbacks=[
        checkpoint_callback,
        early_stop_callback,
        generate_schematic_callback
    ]
)

trainer.fit(lightning_model, datamodule=data_module)

c:\Users\mmmfr\Documents\Repositories\minecraft-schematic-generator\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Seed set to 0
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type | Params
------------------------------
0 | vae  | VAE  | 102 M 
------------------------------
102 M     Trainable params
0         Non-trainable params
102 M     Total params
410.976   Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

c:\Users\mmmfr\Documents\Repositories\minecraft-schematic-generator\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.


c:\Users\mmmfr\Documents\Repositories\minecraft-schematic-generator\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.


...

Epoch 66: 100%|██████████| 42/42 [00:05<00:00,  8.16it/s, v_num=21]

Metric val_loss improved by 0.000 >= min_delta = 0.0. New best score: 0.270


Epoch 86: 100%|██████████| 42/42 [00:04<00:00,  8.41it/s, v_num=21]

Monitored metric val_loss did not improve in the last 20 records. Best score: 0.270. Signaling Trainer to stop.


Epoch 86: 100%|██████████| 42/42 [00:05<00:00,  7.55it/s, v_num=21]


Running Best Model

In [ ]:
import os

import optuna
from lightning.pytorch import Trainer, seed_everything
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.profilers import SimpleProfiler

from common.file_paths import BASE_DIR
from modules import (GenerateSchematicCallback,
                     LightningTransformerMinecraftStructureGenerator,
                     MinecraftDataModule)

study_name = 'study_20231205030812'
storage_url = 'sqlite:///studies.db'

study = optuna.load_study(study_name=study_name, storage=storage_url)


def get_nth_best_trial(study, n, objective_id=0):
    # Sort the completed trials of the study object by the specified objective value.
    sorted_trials = sorted(
        study.trials, key=lambda t: t.values[objective_id] if t.values is not None else float('inf'))

    # Return the parameters of the n-th best trial.
    return sorted_trials[n].params


objective_id = 1
x = 0
nth_best_params = get_nth_best_trial(study, x, objective_id=0)
print(nth_best_params)

seed_everything(1, workers=True)

lightning_model = LightningTransformerMinecraftStructureGenerator(
    num_classes=20,
    max_sequence_length=512,
    freeze_encoder=True,
    **nth_best_params
)

hdf5_file = os.path.join(BASE_DIR, 'data.h5')
data_module = MinecraftDataModule(
    file_path=hdf5_file,
    batch_size=32,
    # num_workers=4
)

logger = TensorBoardLogger(
    'lightning_logs', name='minecraft_structure_generator', log_graph=False)
profiler = SimpleProfiler()
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    mode='min',
    save_last=True
)
early_stop_callback = EarlyStopping(
    monitor='val_loss',
    patience=50,
    verbose=False,
    mode='min'
)
generate_schematic_callback = GenerateSchematicCallback(
    save_path='schematic_viewer/public/schematics/',
    data_module=data_module,
    generate_train=False,
    generate_val=True,
    generate_all_datasets=False,
    generate_every_n_epochs=10,
    autoregressive=True
)

trainer = Trainer(
    max_epochs=5000,
    logger=logger,
    profiler=profiler,
    gradient_clip_val=1.0,
    log_every_n_steps=5,
    callbacks=[
        checkpoint_callback,
        early_stop_callback,
        generate_schematic_callback
    ]
)

trainer.fit(lightning_model, datamodule=data_module)

Seed set to 1


{'embedding_dim': 64, 'embedding_dropout': 0.5, 'decoder_dim': 32, 'num_heads': 2, 'num_layers': 3, 'decoder_dropout': 0.2, 'learning_rate': 0.0006395506262517658}


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                                   | Params
-----------------------------------------------------------------
0 | model | TransformerMinecraftStructureGenerator | 66.9 M
-----------------------------------------------------------------
489 K     Trainable params
66.4 M    Non-trainable params
66.9 M    Total params
267.410   Total estimated model params size (MB)


Epoch 408: 100%|██████████| 21/21 [00:03<00:00,  5.57it/s, v_num=22]


FIT Profiler Report

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  Action                                                                                                                                                               	|  Mean duration (s)	|  Num calls      	|  Total time (s) 	|  Percentage %   	|
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  Total                                                                                                                                                                	|  -     